In [1]:
import numpy as np
import csv
import json
import os 

import pprint 
pp = pprint.PrettyPrinter(indent=4)


In [2]:
import config

In [3]:
#
# Summary log type
#
class SummaryLog:
    """
    Represents one summary log file.
    """
    def __init__(self):
        self.data = []
                
    @staticmethod
    def parse_file(full_path, filename):

        log = SummaryLog()
        fpth = os.path.join(full_path, filename)

        with open(fpth, 'r') as ifs:    
            line = ifs.readline()
            while (line):
                log.parse_line(line)
                line = ifs.readline()

        return log
    
    def parse_line(self, line):
        self.data.append(line)

In [4]:
#
# JSON log type
#
class JsonLog:
    """
    Represents one JSON log file.
    """
    def __init__(self):
        pass
        
    @staticmethod
    def fix_JSON(JSON_text, filename):

        if (len(JSON_text) == 0):
            raise Exception("File {} is empty!".format(filename))

        #
        # Remove an extra comma at the end of the file
        #
        comma_idx = JSON_text.rfind(",")
        text_len = len(JSON_text)
        if (comma_idx >= text_len - 3):
            JSON_text = JSON_text[0:comma_idx] + "\n"


        #
        # Wrap submit logs with an array, cause there may be multiple submits at the same millisecond
        #
        if (filename.endswith("submit.json")):
            JSON_text = "[{}]".format(JSON_text)


        fixed = JSON_text

        try:
            parsed = json.loads(fixed)
            return fixed
        except:
            print(fixed)
            raise Exception("nende")

    @staticmethod
    def parse_file(full_path, filename):
        fpth = os.path.join(full_path, filename)
        parsed = None
        with open(fpth, 'r') as ifs:    
            try:

                parsed = json.load(ifs)
                if (filename.endswith("submit.json") and type(parsed) is dict):
                    raise Exception("!!")
            except:
                ifs.seek(0)
                text = ifs.read()

                parsed = JsonLog.fix_JSON(text, filename)
                ifs.close()

                stinfo = os.stat(fpth)                
                with open(fpth, 'w') as ofs:
                    ofs.write(parsed)
                    print("W: Rewritten the file {}!".format(filename))
                os.utime(fpth ,(stinfo.st_atime, stinfo.st_mtime))
        return parsed      

In [9]:
class Data:
    def __init__(self):
        self.verbose = False
        self.parsed = {}
        
    def process_team(self, team_name: str, team_names: list, verbose = False):
        self.verbose = verbose
        
        print("==============\nTEAM: {}\n".format(team_name))
        
        ###
        #self.validate_and_fix_input_data(team_name, team_names)
        self.calculate_server_ts_diff(team_name, team_names)
        self.generate_DRES_logs(team_name, team_names)
        
        ###
        
        self.verbose = False
     
    
    def validate_and_fix_input_data(self, team_name: str, team_names: list):
        print("%%% VALIDATING & FIXING %%%")
            
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            self.parsed[team_name] = self.validate_user(user_name, path)
            
        print("%%% DONE! %%%")
    
    def calculate_server_ts_diff(self, team_name: str, team_names: list):
        print("%%% CALCULATING SERVER TS DIFF %%%")
            
        diffs = []
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            ds = self.calculate_server_ts_diff_for_user(user_name, path)
            
            diffs.append(ds)
            
        
        #pp.pprint(diffs)
        
        mins = []
        
        for dfs in diffs:
            mins.append(np.min(np.array(dfs)))
            
        i = 0
        for user_name in team_names:
            print("DIFF MIN FOR {}: {}".format(user_name, mins[i]))
            i += 1
            
        print("%%% DONE! %%%")    
        
        return mins
        
    def generate_DRES_logs(self, team_name: str, team_names: list):
        print("%%% GENERATING DRES LOG FILES %%%")
            
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            self.generate_DRES_results_for_user(team_name, user_name, path)
            
        print("%%% DONE! %%%")
        

    
    def calculate_server_ts_diff_for_user(self, user_name, path):
        dir = config.dir_names()["actions"]
            
        diffs = []
        
        if (self.verbose):
            print("\t--- DIR: {} ---".format(dir))

        full_path = os.path.join(path, dir)
        
        for filename in os.listdir(full_path): 
            actions = JsonLog.parse_file(full_path, filename)
            
            for a in actions:
                ser_ts = a["metadata"]["serverTimestamp"]
                loc_ts = a["metadata"]["timestamp"]
                diff = ser_ts - loc_ts
                
                if ser_ts < 10000:
                    continue
                
                diffs.append(diff)

        if (self.verbose):
            print("\t--- DONE. ---")
        
            
        return diffs
    
    def generate_DRES_results_for_user(self, team_name, user_name, path):
        dir = config.dir_names()["requests"]
        out_dir = config.out_dir("dres")
        
        
            
        if (self.verbose):
            print("\t--- DIR: {} ---".format(dir))
            print("\t--- OUT: {} ---".format(out_dir))

        full_path = os.path.join(path, dir)
        
        # For each file in the 'requests' directory
        for filename in os.listdir(full_path): 
            if (not filename.endswith("result.json")):
                continue
                
            file_json = JsonLog.parse_file(full_path, filename)
            
            ts = file_json["timestamp"]
            file_json["request"]["timestamp"] = ts
            
            user_out_dir = os.path.join(out_dir, team_name, user_name)
            os.makedirs(user_out_dir, exist_ok = True)
            
            fpth = os.path.join(user_out_dir, "{}.json".format(ts))
            
            stinfo = os.stat(fpth)                
            with open(fpth, "w") as ofs:
                ofs.write(json.dumps(file_json["request"], indent = 4))
                
            os.utime(fpth ,(stinfo.st_atime, stinfo.st_mtime))
                
        if (self.verbose):
            print("\t--- DONE. ---")
        
    def validate_user(self, user_name, path):
        for _, dir in config.dir_names().items():
            
            if (self.verbose):
                print("\t--- DIR: {} ---".format(dir))
                
            full_path = os.path.join(path, dir)
            for filename in os.listdir(full_path): 
                
                if (filename.endswith("log")):
                     SummaryLog.parse_file(full_path, filename)   
                else:
                    JsonLog.parse_file(full_path, filename)
                    
            if (self.verbose):
                print("\t--- DONE. ---")
                
                        

In [10]:
data = Data()
data.process_team("SOMHunter", ["sh-patrik", "sh-vit"], verbose=True)


TEAM: SOMHunter

%%% CALCULATING SERVER TS DIFF %%%
---
	 +++ sh-patrik +++ 
	DATA: ./data/sh-patrik 

	--- DIR: actions ---
	--- DONE. ---
---
	 +++ sh-vit +++ 
	DATA: ./data/sh-vit 

	--- DIR: actions ---
	--- DONE. ---
DIFF MIN FOR sh-patrik: -123
DIFF MIN FOR sh-vit: 1
%%% DONE! %%%
%%% GENERATING DRES LOG FILES %%%
---
	 +++ sh-patrik +++ 
	DATA: ./data/sh-patrik 

	--- DIR: eval-server-requests ---
	--- OUT: ./out/dres ---


FileNotFoundError: [WinError 2] The system cannot find the file specified: './out/dres\\SOMHunter\\sh-patrik\\1624275277442.json'